In [ ]:
import warnings
warnings.filterwarnings("ignore") 
import arviz as az
import numpy as np
import pandas as pd
import pickle
from typing import List, Dict, Any

# Set Plotly as default plotting backend for pandas
pd.options.plotting.backend = "plotly"  

from tb_incubator.constants import set_project_base_path, ImplementCDR, QUANTILES
from tb_incubator.scenario_utils import (
    calculate_xpert_scenario_outputs,
    calculate_detection_scenario_outputs,
    calculate_acf_scenario_outputs,
    calculate_tsr_scenario_outputs,
    calculate_progressive_intervention_scenarios,
    calculate_combined_scenario_diff_cum_quantiles
)

from tb_incubator.utils import create_periodic_time_series

# Set project base path
project_paths = set_project_base_path("../tb_incubator/")


In [ ]:
calib_out = project_paths["OUT_PATH"]
output_path = project_paths["OUTPUTS"]
file_suffix = "cdr_notif_25000d10000t_01rs"

In [ ]:
# extract data - only run for first time
idata = az.from_netcdf(calib_out / f'calib_full_out_{file_suffix}.nc')

In [ ]:
burnt_idata = idata.sel(draw=np.s_[5000:])
idata_extract = az.extract(burnt_idata, num_samples=1000)
inference_data = az.convert_to_inference_data(idata_extract.reset_index('sample'))
az.to_netcdf(inference_data, calib_out / f'inference_data_{file_suffix}.nc')

In [ ]:
# Load saved idata
idata = az.from_netcdf(calib_out / f'inference_data_{file_suffix}.nc')

In [ ]:
params = {
    "start_population_size": 1.0,
    "seed_time": 1805.0,
    "seed_num": 1.0,
    "seed_duration": 1.0,
    "acf_sensitivity": 0.90
}

# Increase in Xpert utilisation (70% - 95%)

In [ ]:
basecase_xpert = calculate_xpert_scenario_outputs(params, idata, covid_effects=True, apply_cdr=ImplementCDR.ON_NOTIFICATION,
                                                  xpert_target_list=[0.7, 0.8, 0.95])
with open(output_path / f'xpert_quant_outputs.pkl', 'wb') as f:
     pickle.dump(basecase_xpert, f)

# Increase in passive detection (2 - 5 fold increase)

In [ ]:
basecase_detection = calculate_detection_scenario_outputs(params, idata, covid_effects=True, apply_cdr=ImplementCDR.ON_NOTIFICATION, 
                                                          detection_multiplier_list=[2.0, 3.0, 5.0])
with open(output_path / f'detection_quant_outputs.pkl', 'wb') as f:
     pickle.dump(basecase_detection, f)

# Increase in treatment success rate (90% - 95%)

In [ ]:
basecase_tsr = calculate_tsr_scenario_outputs(params, idata, covid_effects=True, apply_cdr=ImplementCDR.ON_NOTIFICATION, 
                                                          tsr_target_list=[0.90, 0.95])
with open(output_path / f'tsr_quant_outputs.pkl', 'wb') as f:
     pickle.dump(basecase_tsr, f)

# ACF Implementation

In [ ]:
percentages = [1, 5, 10, 20, 30, 40, 50, 60]
acf_configs = {
    f"acf_{pct}": create_periodic_time_series(2026., pct/100, 1.)
    for pct in percentages
}


In [ ]:
basecase_acf = calculate_acf_scenario_outputs(params, idata, acf_rate_configs=acf_configs, frequency=None, covid_effects=True, apply_cdr=ImplementCDR.ON_NOTIFICATION)

with open(output_path / f'acf_quant_outputs.pkl', 'wb') as f:
     pickle.dump(basecase_acf, f)

# Combinations

## Progressive scenario

In [ ]:
periodic_acf = create_periodic_time_series(2026.0, 0.6, 1.0)

model_configs = {
    'xpert': {
        'xpert_util_target': 0.95,
        'tsr_target': None,
        'improved_detection_multiplier': None,
        'acf_screening_rate': None,
    },
    'xpert_tsr': {
        'xpert_util_target': 0.95,
        'tsr_target': 0.95,
        'improved_detection_multiplier': None,
        'acf_screening_rate': None,
    },
    'xpert_tsr_detect': {
        'xpert_util_target': 0.95,
        'tsr_target': 0.95,
        'improved_detection_multiplier': 5.,
        'acf_screening_rate': None,
    },
    'xpert_tsr_detect_acf': {
        'xpert_util_target': 0.95,
        'tsr_target': 0.95,
        'improved_detection_multiplier': 3.,
        'acf_screening_rate': periodic_acf,
    },
}

In [ ]:
progressive_scenario_best = calculate_progressive_intervention_scenarios(params, idata, covid_effects=True,
                                                                    apply_cdr=ImplementCDR.ON_NOTIFICATION, 
                                                                    xpert_util_target=0.95,
                                                                    tsr_target=0.95,
                                                                    improved_detection_multiplier=5.,
                                                                    acf_frequency=1.,
                                                                    acf_rate=0.6, baseline_year=2026.0)

In [ ]:
with open(output_path / f'progressive_quant_outputs.pkl', 'wb') as f:
     pickle.dump(progressive_scenario_best, f)

In [ ]:
progressive_scenario_low = calculate_progressive_intervention_scenarios(params, idata, covid_effects=True,
                                                                    apply_cdr=ImplementCDR.ON_NOTIFICATION, 
                                                                    xpert_util_target=0.70,
                                                                    tsr_target=0.90,
                                                                    improved_detection_multiplier=2.,
                                                                    acf_frequency=1.,
                                                                    acf_rate=0.01)

In [ ]:
with open(output_path / f'progressive_quant_outputs_low.pkl', 'wb') as f:
     pickle.dump(progressive_scenario_low, f)

In [ ]:
progressive_scenario_mid = calculate_progressive_intervention_scenarios(params, idata, covid_effects=True,
                                                                    apply_cdr=ImplementCDR.ON_NOTIFICATION, 
                                                                    xpert_util_target=0.80,
                                                                    tsr_target=0.90,
                                                                    improved_detection_multiplier=3.,
                                                                    acf_frequency=1.,
                                                                    acf_rate=0.30)

In [ ]:
with open(output_path / f'progressive_quant_outputs_mid.pkl', 'wb') as f:
     pickle.dump(progressive_scenario_mid, f)

In [ ]:
progressive_scenario_oth = calculate_progressive_intervention_scenarios(params, idata, covid_effects=True,
                                                                    apply_cdr=ImplementCDR.ON_NOTIFICATION, 
                                                                    xpert_util_target=0.70,
                                                                    tsr_target=0.90,
                                                                    improved_detection_multiplier=5.,
                                                                    acf_frequency=1.,
                                                                    acf_rate=0.30)

In [ ]:
with open(output_path / f'progressive_quant_outputs_oth.pkl', 'wb') as f:
     pickle.dump(progressive_scenario_oth, f)

## Cumulative-progressive

In [ ]:
periodic_acf = create_periodic_time_series(2026.0, 0.30, 1.)

model_configs = {
    'acf': {
        'xpert_util_target': None,
        'tsr_target': None,
        'improved_detection_multiplier': None,
        'acf_screening_rate': periodic_acf,
    },
    'acf_xpert': {
        'xpert_util_target': 0.70,
        'tsr_target': None,
        'improved_detection_multiplier': None,
        'acf_screening_rate': periodic_acf,
    },
    'acf_xpert_tsr': {
        'xpert_util_target': 0.70,
        'tsr_target': 0.90,
        'improved_detection_multiplier': None,
        'acf_screening_rate': periodic_acf,
    },
    'acf_xpert_tsr_detect': {
        'xpert_util_target': 0.70,
        'tsr_target': 0.90,
        'improved_detection_multiplier': 5.,
        'acf_screening_rate': periodic_acf,
    },
}

In [ ]:
cumul_scenario2 = calculate_combined_scenario_diff_cum_quantiles(params, idata, model_configs, apply_diagnostic_capacity=True, cumulative_start_time=2025, year=2029.0)
with open(output_path / f'cumul_progress2.pkl', 'wb') as f:
     pickle.dump(cumul_scenario2, f)